In [1]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra
using Distributions

include("COVIDResourceAllocation.jl")
using .COVIDResourceAllocation

In [2]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI"]

start_date = Date(2020, 5, 1)
end_date   = Date(2020, 5, 30)

pct_beds_available = 0.25
travel_threshold_hours = 4.0
hospitalized_days = 14;

In [3]:
N = length(states);
T = (end_date - start_date).value + 1;

In [4]:
forecast_admitted = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
);

In [5]:
forecast_initial = forecast(
    states, start_date-Dates.Day(1), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
);[:];

In [6]:
forecast_discharged = forecast(
    states, start_date-Dates.Day(hospitalized_days), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = hcat(forecast_discharged, zeros(Float32, N, T - hospitalized_days));

In [7]:
beds = n_beds(states, bed_type=:all, pct_beds_available=pct_beds_available);
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours);

In [8]:
pct_nurses_available = 0.5
patients_per_nurse_covid = 2.5
nurse_hrs_per_week_covid = 36

@show nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7
@show nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid

@show nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid;
forecast_active = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
);
nurses = n_nurses(states) * Float32(pct_nurses_available);

nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7 = 5.142857142857143
nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid = 4.666666666666666
nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid = 1.8666666666666665


In [9]:
isolation_spot = [7,8]
model = patient_nurse_allocation_new(
    beds,
    forecast_initial,
    forecast_discharged,
    forecast_admitted,
    nurses,
    adj,
    los=Exponential(100),
    isolation_spot = isolation_spot, 
    verbose=true
)
sentp = value.(model[:sentnurses])
sentn = value.(model[:sentpatients])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

MethodError: MethodError: no method matching patient_nurse_allocation_new(::Vector{Float32}, ::Matrix{Float32}, ::Matrix{Float32}, ::Matrix{Float32}, ::Vector{Float32}, ::BitMatrix; los::Exponential{Float64}, isolation_spot::Vector{Int64}, verbose::Bool)

Closest candidates are:
  patient_nurse_allocation_new(::Vector{<:Real}, !Matched::Vector{<:Real}, ::Matrix{<:Real}, ::Matrix{<:Real}, ::Vector{<:Real}, ::BitMatrix; los, nurse_days_per_patient_day, sendreceive_switch_time, min_send_amt, smoothness_penalty, setup_cost, sent_penalty, balancing_thresh_patients, balancing_penalty_patients, nurse_target_load, nurse_target_load_gap, nurse_load_penalty, disallow_nurse_shortage_sent, disallow_nurse_shortage_newpatients, severity_weighting, isolation_spot, no_artificial_overflow, no_artificial_shortage, verbose)
   @ Main.COVIDResourceAllocation ~/Desktop/Healthcare-Resources-Optimization-main/models/PatientNurseAllocationNew.jl:16


In [11]:
typeof(isolation_spot)

Vector{Int64} (alias for Array{Int64, 1})